# Performing differential expression analysis with AlphaQuant and investigating the results

Below some code how to perform differential expression testing with AlphaQuant. This means we do a pairwise comparison of biological conditions and some volcano plot visualizations.

## Specifying input files

In order to run differential analyses you need:
* an input file from a proteomics search engine (e.g. DIA-NN, Spectronaut, AlphaPept etc.), or a pre-formatted file
* a sample mapping file that maps each sample to a condition (e.g.  sample 'brain_replicate_1' is mapped to condition 'brain'). In the GUI, there is some functionality to help create such a file
* (optional) a results directory can be defined on where to save the data
* (optional) a list where we specify, which conditions we compare

AlphaQuant is compatible to the default output tables of most common proteomics search engines. Detailed specifications on which tables you need can be found 
in our  [README](https://github.com/MannLabs/alphaquant/blob/main/README.md#section-title).


In [ ]:
INPUT_FILE = "./data/mouse_tissues/mouse_report_example_nb_subset.tsv"
SAMPLEMAP_FILE = "./data/mouse_tissues/samplemap_200.tsv"
CONDPAIRS_TO_COMPARE = [('spleen', 'testis'), ('lungs', 'brain')]
RESULTS_DIRECTORY = "./data/mouse_tissues/results_pairwise_analysis"

In [ ]:
#displaying the samplemap file
import pandas as pd
display(pd.read_csv(SAMPLEMAP_FILE, sep='\t'))


## Run AlphaQuant in pairwise-comparison mode

Now we can use the 'run_pipeline' command from AlphaQuant. This performs normalization between sample and then differential expression testing. The main output is a table of proteins with fold changes and an fdr value as a result of the differential expression test. Due to AlphaQuant's tree based approach, sensitivity is in general higher as compared to standard approaches like the t-test, i.e. there should be more proteins with significant fdr, especially for lower fold changes.

In [ ]:
import alphaquant.run_pipeline as aq_pipeline

aq_pipeline.run_pipeline(input_file=INPUT_FILE, samplemap_file=SAMPLEMAP_FILE,
                         condpairs_list=CONDPAIRS_TO_COMPARE, results_dir=RESULTS_DIRECTORY, runtime_plots=True,
						 cluster_threshold_pval=0.2)#CAUTION: we simulate a proteoform candidate in this small dataset by setting the cluster_threshold_pval to 0.2 which is very permissive and allows for random hits. We recommend using the default value (0.001) otherwise

## Inspecting output tables

The main output tables for each pairwise comparison are located in the results folder and are named
 * spleen_VS_testis.results.tsv
 * lungs_VS_brain.results.tsv

Let's load one of them:
 
 

In [ ]:
import pandas as pd

df_spleen_testis  = pd.read_csv(RESULTS_DIRECTORY + "/spleen_VS_testis.results.tsv", sep='\t')
display(df_spleen_testis)

The fold changes are defined as condition1 - condition2. For example spleen_VS_testis means that a log2fc is calculated by log2(intensity_spleen/intensity_testis)

In [ ]:
df_spleen_testis["abs_fc"] = abs(df_spleen_testis["log2fc"])
display(df_spleen_testis.sort_values(by='abs_fc').iloc[:15])

## Visualizing results - basic plots

### Volcano plot

In [ ]:
import alphaquant.plotting.pairwise as aq_plotting_pairwise

aq_plotting_pairwise.volcano_plot(df_spleen_testis)

### Normalization plot

The normalization plot gives the foldchanges between a given sample and the median over all samples. The peak of the distributions should be centered around 0. If not, there might be a problem with the normalization.

In [ ]:
normalized_df = pd.read_csv(RESULTS_DIRECTORY + "/spleen_VS_testis.normed.tsv", sep='\t')
samplemap_df = pd.read_csv(SAMPLEMAP_FILE, sep='\t')

In [ ]:
aq_plotting_pairwise.plot_normalization_overview(normalized_df, samplemap_df)

## Inspect and visualize proteoforms

Peptides that show different quantititative behaviour than other peptides can indicate proteoforms. The logic behind this can be elucidated with a simple example. Let's assume we have a protein with 3 peptides in spleen and testis. The protein itself shows no regulation between the two tissues. Therefore, we can expect that neither of the peptides changes between the conditions. The log2 fold changes for all the peptides should be the same. Now, let's assume that one of the peptides is phosphorylated in testis. If fifty percent of the copies of this peptide carry a phosphorylation, then only half as much **non-modified** peptide should be detectable in testis. This means, that in out spleen_vs_testis condition, we expect the unmodified version of the phosphorylated peptide to be up-regulated, while the other peptides have no regulation. The fold changes should differ. With AlphaQuant, we can investigate systematic differences between fold changes and report peptides that behave differently than expected.

We write out a `.proteoforms.tsv` table for each pair of conditions. Let's load the one for spleen and testis:

In [ ]:
import pandas as pd
proteoform_df = pd.read_csv(RESULTS_DIRECTORY + "/spleen_VS_testis.proteoforms.tsv", sep='\t')
display(proteoform_df)

An important column is termed "fcdiff", which quantifies the fold change difference between peptides and groups of peptides. Let's filter for proteoforms with a log2 fold change difference of at least 0.7, using an AlphaQuant function:

In [ ]:
import alphaquant.utils.proteoform_utils as aq_utils_proteoforms

proteoform_df_filtered = aq_utils_proteoforms.filter_proteoform_df(proteoform_df=proteoform_df, min_num_peptides=1)
display(proteoform_df_filtered)

This leaves only a few proteins of interest. Let's visualize them in order to check them out further. For this we must load a few modules in AlphaQuant:

In [ ]:
import alphaquant.plotting.fcviz as aq_plot_fc

proteins_of_interest = proteoform_df_filtered['protein'].unique()
print(proteins_of_interest)
fc_visualizer = aq_plot_fc.FoldChangeVisualizer(condition1="spleen", condition2="testis", results_directory=RESULTS_DIRECTORY, samplemap_file=SAMPLEMAP_FILE)

fc_visualizer.plot_list_of_proteins(proteins_of_interest)

In [ ]:
import alphaquant.plotting.alphamapviz as aq_plot_alphamap

amap_visualizer = aq_plot_alphamap.AlphaMapVisualizer(condition1="spleen", condition2="testis", results_directory=RESULTS_DIRECTORY, samplemap_file=SAMPLEMAP_FILE,protein_identifier='uniprot_id', organism="Mouse") #because we need to map the protein to it's respective sequence, we need to specify that the protein identidier is uniprot and the organism is Mouse

In [ ]:
fc_fig, alphamap_go_fig = amap_visualizer.visualize_protein("Q91VW3")

fc_fig.show()
alphamap_go_fig.show()